Use attract mode and simple chaos monkey to get some embeddings (5 minutes of play) and plot both datasets. Idealy there is no need to change the "action.lua" script. However, the rom_path variable needs to be changed to the path to your rom file.

In [ ]:
%pylab inline
import subprocess
from keras.applications.inception_v3 import InceptionV3
from sklearn.manifold import TSNE
import os

# Attract Mode

In [ ]:
%%time

#Function for saving image and getting embedding
def img_lua():
    img_path = 'temp.png'
    proc.stdin.write(b'client.screenshot("' + bytes(img_path, 'utf-8') + b'");')
    proc.stdin.write(b'io.stdout:write("continue");')
    proc.stdin.flush()
    while proc.stdout.readline() != b'continue\n':
        pass
    
    #Keras only take a batch of data point so a batch of
    #one data point is feeded to keras
    temp_img = np.expand_dims(imread(img_path), axis=0)
    return embedded_model.predict(temp_img)[0]

FRAMES_PER_STEP = 30
ACTION_NUM = 4096 #There are in total 4096 combinations of actions
preparation = False
distribution = np.load('SNES_distribution.npy') #The distribution of button pressed
embedded_model = InceptionV3()
bizhawk_path = 'BizHawk-2.2.2/'
rom_path = '' #Path to the .smc rom file
steps = 600

In [ ]:
%%time

embeddings = []

#Start BizHawk
proc = subprocess.Popen([bizhawk_path + 'EmuHawk.exe',
                         rom_path,
                         '--lua=../action.lua'],
                         stdout=subprocess.PIPE,
                         stdin=subprocess.PIPE)

while True:
    out_line = proc.stdout.readline()
    
    if out_line[:5] == b'start':
        preparation = True
        
    if preparation:
        proc.stdin.write(b'client.speedmode(400);')
        proc.stdin.flush()
        
        #Press no button
        for i in range(steps * FRAMES_PER_STEP):
            if i % FRAMES_PER_STEP == 0:
                embeddings.append(img_lua())
            proc.stdin.write(b'emu.frameadvance();')
            proc.stdin.flush()
        
        embeddings.append(img_lua())
        break
    
    #Terminate EmuHawk
    if out_line == b'':
        break
        
proc.terminate()
attract_embeddings = np.array(embeddings)

# Chaos Monkey

In [ ]:
%%time

embeddings = []

#Start BizHawk
proc = subprocess.Popen([bizhawk_path + 'EmuHawk.exe',
                         rom_path,
                         '--lua=../action.lua'],
                         stdout=subprocess.PIPE,
                         stdin=subprocess.PIPE)

while True:
    out_line = proc.stdout.readline()
    
    if out_line[:5] == b'start':
        preparation = True
        
    if preparation:
        proc.stdin.write(b'client.speedmode(400);')
        proc.stdin.flush()
        
        #SNES
        action_name = ["Up", "Down", "Left", "Right", "Select",
                       "Start", "B", "A", "X", "Y", "L", "R"]
        
        #Press random buttons based on the buttons distribution
        for i in range(steps * FRAMES_PER_STEP):
            if i % FRAMES_PER_STEP == 0:
                embeddings.append(img_lua())
                
                #Select what buttons to press
                action_code = b''
                temp_action = np.random.choice(4096, p=distribution)
                action = "{0:b}".format(temp_action).rjust(12, '0')[::-1]

                action_code += b'buttons = {}'
                for j, name in enumerate(action_name):
                    if action[j] == '1':
                        action_code += b'buttons["' + str.encode(name) + b'"] = 1;'

                action_code += b'joypad.set(buttons, 1);'
                action_code += b'emu.frameadvance();'

                proc.stdin.write(action_code)
                proc.stdin.flush()
            else:
                proc.stdin.write(b'joypad.set(buttons, 1);')
                proc.stdin.write(b'emu.frameadvance();')
                proc.stdin.flush()
        
        embeddings.append(img_lua())
        break
    
    #to terminate program after closing EmuHawk
    if out_line == b'':
        break
        
proc.terminate()
chaos_embeddings = np.array(embeddings)
os.remove('temp.png')

# Visualization

In [ ]:
#Plot bounding box sum and nuclear norm as a function of time

def get_score(embeddings):
    embeddings = np.array(embeddings)
    length = len(embeddings)
    bbox_sum = np.zeros(length-1)
    nuc_norm = np.zeros(length-1)
    for i in range(2, length):
        bbox_sum[i-1] = sum(amax(embeddings[:i], axis=0) - amin(embeddings[:i], axis=0))
        nuc_norm[i-1] = linalg.norm(cov(embeddings[:i].T), ord='nuc')
        
    return bbox_sum, nuc_norm


attract_bbox_sum, attract_nuc_norm = get_score(attract_embeddings)
chaos_bbox_sum, chaos_nuc_norm = get_score(chaos_embeddings)

plot(attract_bbox_sum, label='Attract Mode')
plot(chaos_bbox_sum, label='Chaos Monkey')
title('Bounding Box Sum')
legend()
show()

figure()
plot(attract_nuc_norm, label='Attract Mode')
plot(chaos_nuc_norm, label='Chaos Monkey')
title('Nuclear Norm')
legend()
show()

In [ ]:
#Plot attract mode data and chaos monkey data with tSNE

tsne = TSNE(n_components=2)
tsne.fit(np.concatenate((attract_embeddings, chaos_embeddings)))

attract_tsne = tsne.fit_transform(attract_embeddings)
chaos_tsne = tsne.fit_transform(chaos_embeddings)

scatter(attract_tsne[:, 0], attract_tsne[:, 1], color='r', label='Attract Mode')
scatter(chaos_tsne[:, 0], chaos_tsne[:, 1], color='b', label='Chaos Monkey')
legend()
show()